In [0]:
%run ./00_Setup_Environment

In [0]:
schema = 'adventureworks.silver'

In [0]:
def cast_columns(table, table_name):
    for c in table.columns:
        if "_id" in c:
            table = table.withColumn(c, col(c).cast("int"))

    #print(table_name)
    #table.display()
    table.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{schema}.{table_name}")



In [0]:
def nulls_report(table, table_name):
    nulls = []
    
    # checar nulos por coluna
    for c in table.columns:
        null_count = table.filter(col(c).isNull()).count()
        if null_count > 0:
            nulls.append({
                "table": table_name,
                "column": c,
                "null_count": null_count
            })
    return nulls


In [0]:
def duplicates_report(table, table_name):
    duplicates = []
    
    #checar duplicados
    total = table.count()
    distinct = table.dropDuplicates().count()
    duplicate_count = total - distinct

    if duplicate_count > 0:
        duplicates.append({
            "table": table_name,
            "duplicate_rows": duplicate_count
        })

    return duplicates

In [0]:
silver_table = [table.name for table in spark.catalog.listTables(schema)]
silver_table

In [0]:
for t in silver_table:
    table_read = spark.sql(f"SELECT * FROM {schema}.{t}")
    cast_columns(table_read, t)
    nulls = nulls_report(table_read, t)
    duplicates = duplicates_report(table_read, t)
    
print(f"Nulls: {nulls}")
print(f"Duplicates: {duplicates}")